In [61]:
import pysam 
import pandas as pd
import mappy as mp
import os

# Set up reference genomes (slow)

In [49]:
%%time
cast_path = "/nfs/turbo/umms-indikar/shared/projects/poreC/data/references/fasta/Mus_musculus_casteij.CAST_EiJ_v1.dna.toplevel.fa"
cast_ref = mp.Aligner(cast_path)
print('done!')

done!
CPU times: user 1min 32s, sys: 11.1 s, total: 1min 43s
Wall time: 58.3 s


In [50]:
%%time
s129_path = "/nfs/turbo/umms-indikar/shared/projects/poreC/data/references/fasta/Mus_musculus_129s1svimj.129S1_SvImJ_v1.dna.toplevel.fa"
s129_ref = mp.Aligner(s129_path)
print('done!')

done!
CPU times: user 1min 32s, sys: 10.6 s, total: 1min 43s
Wall time: 58 s


# Monomer Alignment

In [64]:
def align_and_classify_read(read, ref1_aligner, ref2_aligner, min_mapq=20):
    """
    Aligns a single read against two references and returns its classification.
    Handles reads with no sequence.
    """
    seq = read.query_sequence
    
    # If the read has no sequence, classify as unmapped and return
    if not seq:
        return "unmapped", read

    # Align to both references
    hits1 = list(ref1_aligner.map(seq))
    hits2 = list(ref2_aligner.map(seq))

    # Get the best alignment for each reference
    best_hit1 = max(hits1, key=lambda hit: getattr(hit, 'score', 0), default=None)
    best_hit2 = max(hits2, key=lambda hit: getattr(hit, 'score', 0), default=None)

    # Extract alignment metrics
    mapq1 = getattr(best_hit1, 'mapq', 0)
    score1 = getattr(best_hit1, 'score', 0)
    nm1 = getattr(best_hit1, 'NM', float('inf'))

    mapq2 = getattr(best_hit2, 'mapq', 0)
    score2 = getattr(best_hit2, 'score', 0)
    nm2 = getattr(best_hit2, 'NM', float('inf'))

    # Classification logic
    is_mapped1 = mapq1 >= min_mapq
    is_mapped2 = mapq2 >= min_mapq

    classification = "unmapped" # Default classification
    if is_mapped1 and not is_mapped2:
        classification = "haplotype1"
    elif not is_mapped1 and is_mapped2:
        classification = "haplotype2"
    elif is_mapped1 and is_mapped2:
        if score1 > score2 and nm1 < nm2:
            classification = "haplotype1"
        elif score2 > score1 and nm2 < nm1:
            classification = "haplotype2"
        else:
            classification = "ambiguous"
            
    return classification, read

In [65]:
def process_bam_file(input_bam_path, ref1_aligner, ref2_aligner, output_bam_path, region=None):
    """
    Processes a BAM file, creating an index if needed. Classifies and tags 
    reads, writes to a new BAM file, and prints summary statistics.
    """
    # Check for BAM index and create if it doesn't exist
    if region and not os.path.exists(input_bam_path + ".bai"):
        print(f"BAM index not found. Creating index for {input_bam_path}...")
        pysam.index(input_bam_path)
        print("Index created.")

    # Initialize stats counters
    stats = {
        "haplotype1": 0,
        "haplotype2": 0,
        "ambiguous": 0,
        "unmapped": 0,
        "total": 0
    }

    with pysam.AlignmentFile(input_bam_path, "rb", check_sq=False) as infile:
        header = infile.header
        classification_map = {
            "haplotype1": 1,
            "haplotype2": 2,
            "ambiguous": 3,
            "unmapped": 0
        }

        with pysam.AlignmentFile(output_bam_path, "wb", header=header) as outfile:
            reads_to_process = infile.fetch(region=region) if region else infile

            for read in reads_to_process:
                classification, classified_read = align_and_classify_read(read, ref1_aligner, ref2_aligner)
                
                # Update stats
                stats[classification] += 1
                stats["total"] += 1
                
                classified_read.set_tag('HP', classification_map.get(classification, 0), 'i')
                outfile.write(classified_read)

    print(f"Processing complete. Tagged BAM file saved to '{output_bam_path}'")
    
    # Print summary statistics
    print("\n--- Classification Summary ---")
    if stats["total"] > 0:
        for key, value in stats.items():
            if key == "total":
                print(f"Total Reads Processed: {value}")
            else:
                percentage = (value / stats["total"]) * 100
                print(f"{key.capitalize():<12}: {value:>8} ({percentage:.2f}%)")
    else:
        print("No reads were processed.")
    print("----------------------------")

In [ ]:
input_bam = "/nfs/turbo/umms-indikar/shared/projects/poreC/pipeline_outputs/population/library/merged.bam"
output_bam_chr7 = "classified_reads_all_tagged.bam"

# Process only chromosome 7
process_bam_file(input_bam, s129_ref, cast_ref, output_bam_chr7")

In [51]:
# def align_monomer(seq: str, ref, name = None) -> pd.DataFrame:
#     """
#     Align one query sequence to a mappy index and return a DataFrame of hits.
#     Columns match common mappy.AlignedSegment fields.
#     """
#     rows = []
#     for hit in ref.map(seq):  # iterate alignments
#         rows.append({
#             "ctg": hit.ctg,                 # reference contig
#             "r_st": hit.r_st,               # ref start (0-based)
#             "r_en": hit.r_en,               # ref end (exclusive)
#             "q_st": getattr(hit, "q_st", None),
#             "q_en": getattr(hit, "q_en", None),
#             "strand": "-" if getattr(hit, "strand", 1) < 0 else "+",
#             "mapq": getattr(hit, "mapq", None),
#             "score": getattr(hit, "score", None),
#             "mlen": getattr(hit, "mlen", None),   # matched length
#             "blen": getattr(hit, "blen", None),   # alignment block length
#             "NM": getattr(hit, "NM", None),       # edits if available
#             "is_primary": bool(getattr(hit, "is_primary", True)),
#             "cigar": getattr(hit, "cigar_str", None),
#             "name" : name, 
#         })
#     return pd.DataFrame(rows)

In [52]:
# %%time 

# #fpath = "/nfs/turbo/umms-indikar/shared/projects/poreC/pipeline_outputs/population/expanded/annotate/batch02.GRCm39.pe.bam"
# fpath = "/nfs/turbo/umms-indikar/shared/projects/poreC/pipeline_outputs/population/library/merged.bam"
# bam = pysam.AlignmentFile(fpath)

# stop = 1
# count = -1

# # loop through each monomoner
# for read in bam:
#     count += 1
#     if count == stop:
#         break

#     # # print the read name
#     print(f"\n{read.qname=}")

#     # alignment of the monomer both genomes 
#     seq = read.seq
#     h1 = align_monomer(seq, s129_ref, name='S129')
#     h2 = align_monomer(seq, cast_ref, name='CAST')

#     align = pd.concat([h1, h2])
#     print(align)
    
#     # compare hits (h1 vs. h2)
#     # examples:
#         # max alignment_score(H1, H2)
#         # max alignment_length(H1, H2)
#         # min alignment_mismatch(H1, H2)
#         # max alignment_quality(H1, H2)
    
#     # assign --> subread to: h1, h2, or unknown

#     # store assignments


read.qname='52aab340-35f0-4a9a-b718-b3381a32c138'
  ctg    r_st    r_en  q_st  q_en strand  mapq score  mlen  blen  NM  \
0   1    2354    2824     0   473      +    60  None   470   473   3   
0   1  199617  200459     0   841      +    60  None   825   848  23   

   is_primary                                              cigar  name  
0        True                                    277M1I146M2I47M  S129  
0        True  277M1I146M2I62M1D25M1D10M2I7M1I87M2D56M1D31M2D...  CAST  
CPU times: user 5.61 ms, sys: 1.86 ms, total: 7.47 ms
Wall time: 10.8 ms
